# scan different paramters for the first high res registartion
This process takes a lot of time and should be done on a few parameters.


In [25]:
%load_ext autoreload
%autoreload 2
from reg_imports import verify_rounds, hjson, Path, np, tif_imread, tif_imwrite, tqdm, register_lowres, custom_easifish_registration_pipeline


# Load the manifest file
hjson_file = '/home/oamsalem/code/Projects/EZfish/ezfish_pipeline/examples/BIG100.hjson'
full_manifest = hjson.load(open(hjson_file, 'r'))
manifest = full_manifest['data']

output_folder = Path(manifest['base_path']) / manifest['mouse_name'] / 'OUTPUT' / 'HCR' / 'registrations' 
round_to_rounds, reference_round, _ = verify_rounds(full_manifest)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
All files found


### PARAMETERS

In [20]:
# Choose the round to register
HCR_round_to_register = '02'# 02,03...?

assert HCR_round_to_register in round_to_rounds.keys(), f"round {HCR_round_to_register} not found in rounds to register"



### Image parameters 

In [21]:
mov_round = round_to_rounds[HCR_round_to_register]
# File names
HCR_fix_image_path = reference_round['image_path'] # The fix image that all other rounds will be registerd to (include all channels!)
HCR_mov_image_path = mov_round['image_path'] # The image that will be registered to the fix image (include all channels!)

round_folder_name = f"HCR{HCR_round_to_register}_to_HCR{reference_round['round']}"

# resolution of the images
fix_image_spacing = np.array(reference_round['resolution']) # Y,X,Z
mov_image_spacing = np.array(mov_round['resolution']) # Y,X,Z

# spatial downsampling, probably no need to test. (Changed x and y from 3 to 2 for CIM round 5)
red_mut_x = manifest['HCR_to_HCR_params']['red_mut_x']
red_mut_y = manifest['HCR_to_HCR_params']['red_mut_y']
red_mut_z = manifest['HCR_to_HCR_params']['red_mut_z']



#### Load the images

In [22]:
# Take first channel, I always assume that the first channel is what we want to register
fix_highres = tif_imread(HCR_fix_image_path)[:,0].transpose(2,1,0) # take first channlel and transform from Z,X,Y to Y,X,Z
mov_highres = tif_imread(HCR_mov_image_path)[:,0].transpose(2,1,0) # take first channlel and transform from Z,X,Y to Y,X,Z


fix_lowres = fix_highres[::red_mut_y,::red_mut_x,::red_mut_z]
fix_lowres_spacing = fix_image_spacing * np.array([red_mut_y,red_mut_x,red_mut_z])


mov_lowres = mov_highres[::red_mut_y,::red_mut_x,::red_mut_z]
mov_lowres_spacing = mov_image_spacing * np.array([red_mut_y,red_mut_x,red_mut_z])

# Print the dimensions of the high-resolution and low-resolution images
print(f"The dimensions of the high-resolution fixed image are: {fix_highres.shape}")
print(f"The dimensions of the high-resolution moving image are: {mov_highres.shape}")
print(f"The dimensions of the low-resolution moving image are: {mov_lowres.shape}")

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


The dimensions of the high-resolution fixed image are: (1942, 1937, 70)
The dimensions of the high-resolution moving image are: (1943, 1943, 62)
The dimensions of the low-resolution moving image are: (648, 648, 31)


### bigstream parameters (Pick what seems to have been good from 1_scan_lowres_parameters, 1 set)

In [23]:
### Global parameters ###
## These should be scanned in the 1_scan_lowres_parameters notebook, should use only 1 set here##

# radius for local ransac for each spot (in pixles for LOW res image) - how many pixels to look around a spot for match
g_radii_z = [1]
g_radii_xy = [25]

# when searching for blobs for ransac, what is the radius for the search alogorithem (in pixles for low res image)
g_bsize_s_b = [[3,30]]

# what is the minumum correlation for a spot to be considered a match
g_m_tresholds = [0.2] 

### Local parameters ###

# block size for local affine and deformable registration # [[x,y,z]]
blocksize_to_scan = [[50,50,12],[100, 100, 25]]

# Radius for local RANSAC for each spot (in pixels for high res image) - how many pixels to look around a spot for match
radii_z = [1, 5, 10]
radii_xy = [15,25, 50]
bsize_s_b = [[5, 70]]
m_tresholds = [0.2]

# Example configurations
# [z = 3, xy = 15, 3,20 bs, th = 0.2] # RG002
# [z = 1, xy = 25, 3,20 bs, th = 0.2] # CIM132


In [ ]:
c = {'n_workers': 15, 'threads_per_worker':2} # Colab00 has 20 cores, 2 threads per core, leave a little wiggle room, added c to args. Camillo has 40 cores.
from ClusterWrap import cluster as cluster_constructor

success_n = 0
write_dir = output_folder / round_folder_name / 'local_params_scan'
write_dir.mkdir(exist_ok=True,parents=True)
write_directory = str(write_dir)
print(write_directory)

#Change this when ready to proceed with full reg! and change params up top before doing so
no_deform = True

if no_deform:
    write_dir = write_dir.parent / f'{write_dir.name}_nodeform'

njobs = len(g_radii_z)*len(g_radii_xy)*len(g_bsize_s_b)*len(g_m_tresholds)*len(blocksize_to_scan)*len(radii_z)*len(radii_xy)*len(bsize_s_b)*len(m_tresholds)
pbar = tqdm(total=njobs)
for g_radius_z in g_radii_z:
  for g_radius_xy in g_radii_xy:
    for g_bsize_s, g_bsize_b in g_bsize_s_b:
      for g_m_treshold in g_m_tresholds:
        for blocksize in blocksize_to_scan:
          for radius_z in radii_z:
            for radius_xy in radii_xy:
              for bsize_s, bsize_b in bsize_s_b:
                for m_treshold in m_tresholds:
                        
                  # wrong folder nameing need to replace the name
                  local_folder = fr"rz{g_radius_z}_rxy{g_radius_xy}_bs{g_bsize_s}to{g_bsize_b}_mt{g_m_treshold}"
                  global_folder = fr"bs{blocksize[0]}_{blocksize[1]}_{blocksize[2]}_{radius_z}_{radius_xy}_bs{bsize_s}to{bsize_b}_mt{m_treshold}"

                  write_directory = write_dir / local_folder / global_folder
                  write_directory.mkdir(exist_ok=True,parents=True)
                  write_directory = str(write_directory); print(write_directory)

                  global_ransac_kwargs = {'cc_radius':(g_radius_xy,g_radius_xy,g_radius_z),
                                          'blob_sizes':[g_bsize_s, g_bsize_b],
                                          'match_threshold':g_m_treshold,
                                          'safeguard_exceptions':False}

                  local_ransac_kwargs = {'cc_radius':(radius_xy,radius_xy,radius_z),
                                               'blob_sizes':[bsize_s, bsize_b], 
                                               'match_threshold': [m_treshold],
                                               'safeguard_exceptions':False}
                    
                
                  cluster = cluster_constructor(c)
                  affine, deform, aligned = custom_easifish_registration_pipeline(
                          fix_lowres, fix_highres, mov_lowres, mov_highres,
                          fix_lowres_spacing, fix_image_spacing,
                          mov_lowres_spacing, mov_image_spacing,
                          blocksize=blocksize,
                          global_ransac_kwargs=global_ransac_kwargs,
                          local_ransac_kwargs=local_ransac_kwargs,
                          write_directory=write_directory,
                          no_deform=no_deform,
                          overwrite_lowres=False,
                          cluster=cluster)
                  print(f'finished! - {write_directory}')
                  pbar.update(1)
                  
                  tif_imwrite(Path(write_directory)  / 'deformed.tiff', np.array(aligned).transpose(2,1,0))
                  # delete zarr file
                  os.remove(Path(write_directory) / 'deformed.zarr')


/mnt/nasquatch/data/2p/jonna/EASI_FISH/pipeline/BIG100/OUTPUT/HCR/registrations/HCR02_to_HCR01/local_params_scan


  0%|          | 0/18 [00:00<?, ?it/s]

/mnt/nasquatch/data/2p/jonna/EASI_FISH/pipeline/BIG100/OUTPUT/HCR/registrations/HCR02_to_HCR01/local_params_scan_nodeform/rz1_rxy25_bs3to30_mt0.2/bs50_50_12_1_15_bs5to70_mt0.2
{'host': ''}


/mnt/colab/colab_shared/anaconda3/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38319 instead
  warnings.warn(


Cluster dashboard link:  http://10.35.76.107:38319/status


/mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_v2_andermann/bigstream/piecewise_align.py:378: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)
/mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_v2_andermann/bigstream/piecewise_align.py:378: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)
/mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_v2_andermann/bigstream/piecewise_align.py:378: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)
/mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_v2_andermann/bigstream/piecewise_align.py:378: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)
/mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_v2_andermann/bigstream/piecewise_align.py:3

In [ ]:
import os
os.getlogin()

'oamsalem'

In [ ]:
import os
import zarr
from tifffile import imwrite as tiff_imwrite

write_directory = fr"\mnt\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan\rz1_rxy25_bs3to30_mt0.2\bs100_100_25_1_25_bs5to70_mt0.2"
zarr_dir =  fr"\mnt\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan\rz1_rxy25_bs3to30_mt0.2\bs100_100_25_1_25_bs5to70_mt0.2\deformed.zarr"

# Check if the Zarr directory exists
if os.path.isdir(zarr_dir):
    print("Zarr directory exists")

    try:
        # Attempt to load the Zarr data
        data = zarr.open_array(zarr_dir, mode='r')
        print(f"Data loaded with shape: {data.shape}")
        
        if data is not None:
            # Save to TIFF file
            tiff_path = os.path.join(write_directory, "deformed.tiff")
            tiff_imwrite(tiff_path, data.transpose(2, 1, 0))
            print(f"TIFF file written successfully at {tiff_path}")
        else:
            print("Failed to load data. It is None.")
    except Exception as e:
        print(f"An error occurred while processing the Zarr data: {e}")
else:
    print("Zarr directory does not exist.")





In [ ]:
## Make combined TIFFs in new folder to check local parameter search

In [4]:
## If scanning local parameters (WITHOUT DEFORM), go through a master folder and turn all deformed.zarr into TIFFs and combine with original DAPI for ease of comparison.
#In future, save only non-degenerate affines would make this way easier.

import os
import zarr
import tifffile
import numpy as np
from tqdm import tqdm
import SimpleITK as sitk

def get_registration_score(fixed, mov):
    fixed_image = sitk.GetImageFromArray(fixed.astype(np.float32))
    registered_image = sitk.GetImageFromArray(mov.astype(np.float32))
    # Initialize the registration method
    irm = sitk.ImageRegistrationMethod()

    # Set the metric to ANTS Neighborhood Correlation
    irm.SetMetricAsMattesMutualInformation()  # 4 is the radius of the neighborhood

    # Set the fixed and moving images for the metric evaluation
    out = irm.MetricEvaluate(fixed_image,registered_image)
    return out

def find_folders_with_zarr(master_folder):
    zarr_folders = []
    for root, dirs, files in os.walk(master_folder):
        if 'deformed.zarr' in dirs:
            zarr_folders.append(root)
    return zarr_folders

def process_zarr_to_tiff(master_folder, dapi_file, subsample_z =2 ):
    zarr_folders = find_folders_with_zarr(master_folder)
    subfolder_count = len(zarr_folders)

    if subfolder_count == 0:
        print("No folders with 'deformed.zarr' found.")
        return

    print(f"Found {subfolder_count} folders with 'deformed.zarr'")
    print("Folders to process:", zarr_folders)

    output_dir = os.path.join(master_folder, 'tiff_outputs_gpt')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    dapi_data = tifffile.imread(dapi_file)
    dapi_data = dapi_data.transpose(2, 1, 0)
    dapi_data_org = dapi_data.copy()
    dapi_data = dapi_data[:, :, ::subsample_z]

    pbar = tqdm(total=subfolder_count, desc="Processing folders")

    for index, zarr_folder in enumerate(zarr_folders):
        print(f"Processing folder {index + 1} out of {subfolder_count}")
        zarr_path = os.path.join(zarr_folder, 'deformed.zarr')
        subfolder_name = os.path.basename(zarr_folder)

        try:
            data = zarr.load(zarr_path)
#             from IPython import embed; embed()
            reg_score= get_registration_score(data, dapi_data_org)
            reg_score_text = str(np.round(reg_score,3)).replace('-','m')
        
            data = data[:, :, ::subsample_z]

            print(f"Zarr data shape: {data.shape}, DAPI data shape: {dapi_data.shape}")

            if data.shape != dapi_data.shape:
                print(f"Skipping {zarr_path} due to shape mismatch with DAPI data")
                pbar.update(1)
                continue

            combined_data = np.stack((dapi_data, data), axis=-1)
            print(f"Combined data shape before reshaping: {combined_data.shape}")

            tiff_path = os.path.join(output_dir, f"{reg_score_text}_{subfolder_name}_both.tiff")
            combined_data = combined_data.transpose(2, 3, 0, 1)
            tifffile.imwrite(tiff_path, combined_data, photometric='minisblack', imagej=True)

            print(f"Converted {zarr_path} to {tiff_path}")

        except Exception as e:
            print(f"Error processing {zarr_path}: {e}")

        pbar.update(1)
    
        # Add a check to break out of the loop if it stalls.

    pbar.close()

master_folder = fr"\\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2"
dapi_file =fr"\\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\CIM132_SLICE001_HCR1_C1_DAPI.tif"

process_zarr_to_tiff(master_folder, dapi_file)


Found 18 folders with 'deformed.zarr'
Folders to process: ['\\\\bananas\\data\\ezfish\\jonna\\CIM132\\HCR\\240408_CIM132\\output_HCR4_to_HCR1_github\\local_params_scan_nodeform\\rz1_rxy25_bs3to30_mt0.2\\bs50_50_12_10_15_bs5to70_mt0.2', '\\\\bananas\\data\\ezfish\\jonna\\CIM132\\HCR\\240408_CIM132\\output_HCR4_to_HCR1_github\\local_params_scan_nodeform\\rz1_rxy25_bs3to30_mt0.2\\bs50_50_12_1_15_bs5to70_mt0.2', '\\\\bananas\\data\\ezfish\\jonna\\CIM132\\HCR\\240408_CIM132\\output_HCR4_to_HCR1_github\\local_params_scan_nodeform\\rz1_rxy25_bs3to30_mt0.2\\bs100_100_25_1_50_bs5to70_mt0.2', '\\\\bananas\\data\\ezfish\\jonna\\CIM132\\HCR\\240408_CIM132\\output_HCR4_to_HCR1_github\\local_params_scan_nodeform\\rz1_rxy25_bs3to30_mt0.2\\bs100_100_25_10_15_bs5to70_mt0.2', '\\\\bananas\\data\\ezfish\\jonna\\CIM132\\HCR\\240408_CIM132\\output_HCR4_to_HCR1_github\\local_params_scan_nodeform\\rz1_rxy25_bs3to30_mt0.2\\bs100_100_25_1_25_bs5to70_mt0.2', '\\\\bananas\\data\\ezfish\\jonna\\CIM132\\HCR\\24040

Processing folders:   0%|                                                                       | 0/18 [00:00<?, ?it/s]

Processing folder 1 out of 18
Zarr data shape: (1969, 1930, 39), DAPI data shape: (1969, 1930, 39)
Combined data shape before reshaping: (1969, 1930, 39, 2)


Processing folders:   6%|███▍                                                          | 1/18 [03:15<55:29, 195.86s/it]

Converted \\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2\bs50_50_12_10_15_bs5to70_mt0.2\deformed.zarr to \\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2\tiff_outputs_gpt\m0.215_bs50_50_12_10_15_bs5to70_mt0.2_both.tiff
Processing folder 2 out of 18
Zarr data shape: (1969, 1930, 39), DAPI data shape: (1969, 1930, 39)
Combined data shape before reshaping: (1969, 1930, 39, 2)


...

Converted \\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2\bs50_50_12_5_50_bs5to70_mt0.2\deformed.zarr to \\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2\tiff_outputs_gpt\m0.215_bs50_50_12_5_50_bs5to70_mt0.2_both.tiff
Processing folder 17 out of 18
Zarr data shape: (1969, 1930, 39), DAPI data shape: (1969, 1930, 39)
Combined data shape before reshaping: (1969, 1930, 39, 2)


Processing folders:  94%|█████████████████████████████████████████████████████████▌   | 17/18 [39:24<03:18, 198.16s/it]

Converted \\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2\bs50_50_12_5_25_bs5to70_mt0.2\deformed.zarr to \\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2\tiff_outputs_gpt\m0.215_bs50_50_12_5_25_bs5to70_mt0.2_both.tiff
Processing folder 18 out of 18
Zarr data shape: (1969, 1930, 39), DAPI data shape: (1969, 1930, 39)
Combined data shape before reshaping: (1969, 1930, 39, 2)


Processing folders: 100%|█████████████████████████████████████████████████████████████| 18/18 [40:29<00:00, 134.96s/it]

Converted \\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2\bs100_100_25_5_15_bs5to70_mt0.2\deformed.zarr to \\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR4_to_HCR1_github\local_params_scan_nodeform\rz1_rxy25_bs3to30_mt0.2\tiff_outputs_gpt\m0.298_bs100_100_25_5_15_bs5to70_mt0.2_both.tiff
